# Convert Borden Dataset into HuggingFace Dataset Format

In [ ]:
import os
import json

## Copy images

In [ ]:
path_to_train = '/data2/eranario/data/yolo_grl_data/BordenNight/Real/train'
path_to_valid = '/data2/eranario/data/yolo_grl_data/BordenNight/Real/valid'
path_to_test = '/data2/eranario/data/yolo_grl_data/BordenNight/Real/test'

all_source_paths = {'train': path_to_train, 'valid': path_to_valid, 'test': path_to_test}

In [ ]:
path_to_target = '/data2/eranario/data/Active-Learning-Datasets/Borden'

In [ ]:
for key, value in all_source_paths.items():
    source = os.path.join(value, 'images')
    target = os.path.join(path_to_target, key)

    # copy source to target
    os.system('cp -r ' + source + ' ' + target)

# Create metadata file from labels

In [27]:
def convert_yolo_to_metadata(label_dir, output_file, image_width, image_height):
    """
    Convert YOLO label files into a single metadata JSON file suitable for HuggingFace Object Detection Dataset.

    Parameters:
    - label_dir: Directory containing the YOLO label files.
    - output_file: Path where the single metadata JSON file will be saved.
    - image_width: Width of the images corresponding to the label files.
    - image_height: Height of the images corresponding to the label files.
    """
    def yolo_to_bbox(yolo_data, img_width, img_height):
        x_center, y_center, width, height = yolo_data
        x_min = (x_center - width / 2) * img_width
        y_min = (y_center - height / 2) * img_height
        bbox_width = width * img_width
        bbox_height = height * img_height
        return [x_min, y_min, bbox_width, bbox_height]

    metadata_list = []

    for label_id, label_file in enumerate(os.listdir(label_dir)):
        if label_file.endswith('.txt'):
            with open(os.path.join(label_dir, label_file), 'r') as file:
                lines = file.readlines()

            bboxes = []
            categories = []
            for line in lines:
                parts = line.strip().split()
                category_id = int(parts[0])
                bbox = yolo_to_bbox(list(map(float, parts[1:])), image_width, image_height)
                bboxes.append(bbox)
                categories.append(category_id)

            metadata = {
                "image_id": label_id,
                "file_name": label_file.replace('.txt', '.jpg'),
                "width": image_width,
                "height": image_height,
                "objects": {
                    "bbox": bboxes,
                    "categories": categories
                }
            }

            metadata_list.append(metadata)

    # Write all metadata entries to a single JSON file
    with open(output_file, 'w') as json_file:
        for entry in metadata_list:
            json.dump(entry, json_file)
            json_file.write('\n')

    print(f"Conversion completed. Metadata saved to {output_file}")

In [28]:
# get example image width and height
example_image = os.path.join(path_to_target, 'train', os.listdir(os.path.join(path_to_target, 'train'))[0])
example_image_width, example_image_height = os.popen(f'identify -format "%w %h" {example_image}').read().split()
print(f"Example image width: {example_image_width}, height: {example_image_height}")

Example image width: 720, height: 1280


In [29]:
for key, value in all_source_paths.items():
    label_dir = os.path.join(value, 'labels')
    output_dir = os.path.join(path_to_target, key, 'metadata.jsonl')
    convert_yolo_to_metadata(label_dir, output_dir, int(example_image_width), int(example_image_height))

Conversion completed. Metadata saved to /data2/eranario/data/Active-Learning-Datasets/Borden/train/metadata.jsonl
Conversion completed. Metadata saved to /data2/eranario/data/Active-Learning-Datasets/Borden/valid/metadata.jsonl
Conversion completed. Metadata saved to /data2/eranario/data/Active-Learning-Datasets/Borden/test/metadata.jsonl


# Test Custom HuggingFace Dataset

In [30]:
from datasets import load_dataset

In [31]:
path_to_hf_train = os.path.join(path_to_target, 'train')
path_to_hf_valid = os.path.join(path_to_target, 'valid')
path_to_hf_test = os.path.join(path_to_target, 'test')

In [32]:
path_to_hf_train

'/data2/eranario/data/Active-Learning-Datasets/Borden/train'

In [33]:
dataset_train = load_dataset('imagefolder', data_dir=path_to_hf_train, split='train')

Generating train split: 98 examples [00:00, 2275.19 examples/s]


In [34]:
dataset_train

Dataset({
    features: ['image', 'image_id', 'width', 'height', 'objects'],
    num_rows: 98
})